# Generate Representations
In this notebook, we generate the inputs for the machine learning model. At the end of this notebook, we will have computed the features for all available data, and generated the pickle files needed to run these models for any available data.

In [1]:
%matplotlib inline
import sys
from glob import glob
sys.path = glob('/projects/CSC249ADCD01/packages/*') + sys.path
from matplotlib import pyplot as plt
import os
import pandas as pd
import numpy as np
import pickle as pkl
import gzip
from stopping_power_ml.io import get_charge_density_interpolator
from stopping_power_ml.features import LocalChargeDensity, IonIonForce, ProjectedAGNIFingerprints, ProjectileVelocity
from tqdm import tqdm_notebook as tqdm

Configure Parsl

In [2]:
import parsl
from parsl import *
workers = IPyParallelExecutor()
dfk = DataFlowKernel(executors=[workers])

Executors :  {0: <IPP Executor for site:Static_IPP>}


## Load in Dataset
Load in the pickled dataset from `parse-qbox-data`

In [3]:
data = pkl.load(gzip.open('data.pkl.gz', 'rb'))
print('Loaded %d entries'%len(data))

Loaded 50069 entries


Load an example of the simualtion cell

In [4]:
start_frame = pkl.load(open('al_starting_frame.pkl', 'rb'))

## Compute the Representation
We use three kinds of features as input to our machine learning model:

1. *Local Charge Density*: The local electron density at the projectile's current position and several positions in the future and past.
1. *Local Environemnt*: Descriptions of the atom environment around the projectile using the AGNI fingerprints developed by [Botu *et al*](http://pubs.acs.org/doi/10.1021/acs.jpcc.6b10908)
1. *Ion-Ion Repulsion*: The Coulombic repulsion force between the Al atoms and the projectile

### Local Charge Density Features
These features are based on the ground-state electronic structure of Al, as computed using DFT. The original 

In [5]:
%%time
charge = pkl.load(open('density_interp.pkl', 'rb'))

Wall time: 60.6 ms


In [6]:
charge_features = LocalChargeDensity(start_frame, charge, np.linspace(1, -5, 7))

Compute the features. First, we define a parallel function with Parsl

In [7]:
def compute_features(featurizer, data, chunks=96):
    """Compute the features for each entry in the dataframe.
    
    :param featurizer: tool used to compute the features
    :param data: pd.DataFrame, data to be featurized
    :param threads_per_worker: int, number of threads to run on each Parsl worker
    :param chunks: int, number of pieces to evaluate in parallel"""
    
    @App('python', dfk)
    def compute_features(chunk):
        return featurizer.featurize_dataframe(chunk, ['position', 'velocity'], inplace=False)

    chunks = [compute_features(chunk) for chunk in np.array_split(data, chunks)]
    return pd.concat([c.result() for c in tqdm(chunks, desc='Running')])

In [8]:
%%time
data = compute_features(charge_features, data)

HBox(children=(IntProgress(value=0, description='Running', max=96), HTML(value='')))


Wall time: 1min 37s


## Compute AGNI Fingerprints
Use these to describe the local enviroment around a particle

In [9]:
agni_features = ProjectedAGNIFingerprints(start_frame, etas=np.logspace(np.log10(0.8), np.log10(16), 8).tolist(),
                                          use_prim_cell=False)

In [10]:
%%time
#data = agni_features.featurize_dataframe(data, 'atoms')
data = compute_features(agni_features, data) # Turned off while writing this notebook

HBox(children=(IntProgress(value=0, description='Running', max=96), HTML(value='')))


Wall time: 1min 8s


## Compute Ion-Ion Interaction Forces
Get the repulsion force due to the ion-ion repulsion from the surrounding Al nuclei. Specifically, we compute the force acting on the particle in the direction of travel using the [Ewald summation method](https://en.wikipedia.org/wiki/Ewald_summation).

In [11]:
ewald_features = IonIonForce(start_frame, acc=4)

In [12]:
%%time
data = compute_features(ewald_features, data) 

HBox(children=(IntProgress(value=0, description='Running', max=96), HTML(value='')))


Wall time: 41 s


## Add Velocity as Feature
Just the magnitude of the projectile velocity

In [13]:
velocity_feature = ProjectileVelocity(start_frame)

In [14]:
%%time
data = compute_features(velocity_feature, data) 

HBox(children=(IntProgress(value=0, description='Running', max=96), HTML(value='')))


Wall time: 4.23 s


## Save everything to disk
Data and featurizers

In [15]:
data.columns

Index(['energy', 'file_id', 'force', 'frame_id', 'position', 'velocity',
       'file', 'timestep', 'displacement', 'directory', 'log density t=1.0',
       'log density t=0.0', 'log density t=-1.0', 'log density t=-2.0',
       'log density t=-3.0', 'log density t=-4.0', 'log density t=-5.0',
       'AGNI projected eta=8.00e-01', 'AGNI projected eta=1.23e+00',
       'AGNI projected eta=1.88e+00', 'AGNI projected eta=2.89e+00',
       'AGNI projected eta=4.43e+00', 'AGNI projected eta=6.80e+00',
       'AGNI projected eta=1.04e+01', 'AGNI projected eta=1.60e+01',
       'AGNI eta=8.00e-01', 'AGNI eta=1.23e+00', 'AGNI eta=1.88e+00',
       'AGNI eta=2.89e+00', 'AGNI eta=4.43e+00', 'AGNI eta=6.80e+00',
       'AGNI eta=1.04e+01', 'AGNI eta=1.60e+01', 'ion-ion repulsion',
       'velocity_mag'],
      dtype='object')

In [16]:
data.to_pickle(gzip.open('training_data.pkl.gz', 'wb'))

In [17]:
pkl.dump([charge_features, agni_features, ewald_features, velocity_feature], open('featurizers.pkl', 'wb'))

In [18]:
dfk.cleanup()